# Chapter 39: TypeScript Compilation Performance

As TypeScript codebases grow, compilation performance becomes a critical factor in developer productivity and CI/CD pipeline efficiency. Slow builds frustrate developers, lengthen feedback loops, and increase infrastructure costs. This chapter explores strategies to optimize TypeScript compilation performance while maintaining type safety and code quality.

---

## 39.1 Understanding Compilation Bottlenecks

Before optimizing, it's essential to understand what makes TypeScript compilation slow and how to diagnose performance issues.

### How TypeScript Compilation Works

TypeScript compilation involves three main phases:

1. **Parsing**: Reading source files and building an Abstract Syntax Tree (AST)
2. **Binding**: Creating symbols and checking scopes
3. **Type Checking**: Resolving types, checking assignments, and validating operations
4. **Emitting**: Writing JavaScript and declaration files

The type checking phase is typically the slowest, as it requires resolving complex type relationships across the entire program.

### Diagnosing Performance Issues

**Enable Performance Tracing:**

```json
// tsconfig.json
{
  "compilerOptions": {
    "generateTrace": "./trace",  // Output performance trace
    "listEmittedFiles": true,    // List all emitted files
    "listFiles": true,           // List all included files
    "explainFiles": true,        // Explain why files are included
    "extendedDiagnostics": true  // Show detailed timing
  }
}
```

**Analyze Compilation Time:**

```bash
# Get detailed timing information
npx tsc --extendedDiagnostics

# Typical output:
# Files:              1250
# Lines:            185000
# Identifiers:      450000
# Symbols:         1250000
# Types:            325000
# Memory used:     485000K
# Parse time:        2.5s
# Bind time:         1.2s
# Check time:        8.5s  # Usually the bottleneck
# Emit time:         3.0s
# Total time:       15.2s
```

**Trace Analysis Tools:**

```bash
# Generate trace
npx tsc --generateTrace ./trace-output

# Analyze with @typescript/analyze-trace
npm install -g @typescript/analyze-trace
analyze-trace ./trace-output

# Identifies:
# - Expensive types causing check time issues
# - Module resolution bottlenecks
# - Files with high instantiation counts
```

### Common Bottlenecks

```typescript
// 1. Excessive Generic Instantiation
// Avoid deeply nested generics that create combinatorial explosions
type DeepNested<T> = T extends object 
  ? { [K in keyof T]: DeepNested<T[K]> } 
  : T;

// With large interfaces, this creates thousands of type instantiations
interface HugeInterface { /* 100+ properties */ }
type Result = DeepNested<HugeInterface>; // Slow!

// 2. Excessive Union Types
// Large unions slow down type checking
type Status = 'pending' | 'active' | 'inactive' | /* ... 50 more */ ;
function processStatus(status: Status) {} // Each call checks all 50+ types

// 3. Complex Conditional Types
type ExtractComplex<T> = T extends { data: infer D }
  ? D extends Array<infer I>
    ? I extends { value: infer V }
      ? V
      : never
    : never
  : never;

// 4. Declaration File Bloat
// Importing huge @types packages increases parsing time
import * as _ from 'lodash'; // If @types/lodash has 10k+ lines
```

---

## 39.2 Project References for Monorepos

Project References allow TypeScript to compile large codebases incrementally by splitting them into smaller projects with explicit dependencies.

### Setting Up Project References

**Monorepo Structure:**
```
monorepo/
├── packages/
│   ├── core/                 # Independent package
│   │   ├── src/
│   │   ├── package.json
│   │   └── tsconfig.json     # tsconfig for core
│   ├── utils/                # Depends on core
│   │   ├── src/
│   │   ├── package.json
│   │   └── tsconfig.json
│   └── app/                  # Depends on core and utils
│       ├── src/
│       ├── package.json
│       └── tsconfig.json
├── tsconfig.json             # Root config (solution file)
└── package.json
```

**Root tsconfig.json (Solution File):**
```json
{
  "compilerOptions": {
    "composite": true,           // Required for project references
    "declaration": true,         // Generate .d.ts files
    "declarationMap": true,      // Source maps for declarations
    "sourceMap": true,
    "target": "ES2022",
    "module": "NodeNext",
    "strict": true,
    "esModuleInterop": true,
    "skipLibCheck": true
  },
  "references": [
    { "path": "./packages/core" },
    { "path": "./packages/utils" },
    { "path": "./packages/app" }
  ],
  "files": []  // Root doesn't contain source files
}
```

**Package tsconfig.json (e.g., packages/core):**
```json
{
  "extends": "../../tsconfig.base.json",
  "compilerOptions": {
    "composite": true,           // Mark as project reference
    "outDir": "./dist",
    "rootDir": "./src",
    "tsBuildInfoFile": "./dist/.tsbuildinfo"
  },
  "include": ["src/**/*"],
  "exclude": ["dist", "node_modules", "**/*.test.ts"]
}
```

**packages/utils/tsconfig.json (with dependency):**
```json
{
  "extends": "../../tsconfig.base.json",
  "compilerOptions": {
    "composite": true,
    "outDir": "./dist",
    "rootDir": "./src"
  },
  "references": [
    { "path": "../core" }        // Explicit dependency
  ],
  "include": ["src/**/*"]
}
```

### Building with Project References

```bash
# Build entire project graph
npx tsc --build

# Incremental build (only changed projects)
npx tsc --build --force

# Watch mode for all projects
npx tsc --build --watch

# Build specific project and dependencies
npx tsc --build packages/app

# Clean build outputs
npx tsc --build --clean
```

**Package.json Scripts:**
```json
{
  "scripts": {
    "build": "tsc --build",
    "build:watch": "tsc --build --watch",
    "build:force": "tsc --build --force",
    "clean": "tsc --build --clean && rm -rf packages/*/dist"
  }
}
```

### Benefits of Project References

```bash
# Without project references: 45 seconds
npx tsc --extendedDiagnostics
# Check time: 35s (checking entire codebase)

# With project references: First build 45s, incremental 3s
npx tsc --build
# Changes in app only: Check time 2s (only rechecking app, using core/utils .d.ts)
```

**Key Advantages:**
1. **Incremental Compilation**: Only rebuilds changed projects and dependents
2. **Parallelization**: Build tools can compile independent projects in parallel
3. **Encapsulation**: Enforces architectural boundaries between packages
4. **IDE Performance**: TypeScript language service loads projects lazily

---

## 39.3 Incremental Compilation

Incremental compilation tracks changes between builds to avoid re-checking unchanged code.

### Configuration

```json
{
  "compilerOptions": {
    "incremental": true,              // Enable incremental compilation
    "tsBuildInfoFile": "./.tsbuildinfo", // Cache file location
    
    // Project references imply incremental
    "composite": true,                // Enables incremental + declaration
    
    // Additional optimizations
    "assumeChangesOnlyAffectDirectDependencies": true  // Faster but less thorough
  }
}
```

### How Incremental Compilation Works

```typescript
// .tsbuildinfo file (JSON) tracks:
{
  "program": {
    "fileInfos": {
      "src/index.ts": {
        "version": "1234567890abcdef",  // Hash of content
        "signature": "fedcba0987654321" // Hash of exported types
      },
      "src/utils.ts": {
        "version": "abcdef1234567890",
        "affectsGlobalScope": true
      }
    },
    "options": {
      "composite": true,
      "declaration": true
    },
    "referencedMap": {
      "src/index.ts": ["src/utils.ts", "src/types.ts"]
    },
    "exportedModulesMap": {
      "src/utils.ts": ["src/types.ts"]
    }
  },
  "version": "5.3.2"
}
```

**Build Process:**
1. First build: Full compilation, generates `.tsbuildinfo`
2. Change `src/index.ts`: TypeScript checks if `src/utils.ts` signature changed
3. If signature unchanged: Only recheck `src/index.ts` using cached type info from `utils.d.ts`
4. If signature changed: Recheck all files importing `utils.ts`

### Limitations and Best Practices

```json
// Avoid these patterns that break incrementality
{
  "compilerOptions": {
    // Don't use with incremental (changes on every build)
    "removeComments": false,  // OK
    // "removeComments": true,  // Problematic with some transformers
    
    // Const enums break incremental if used across projects
    "preserveConstEnums": true,
    
    // Avoid path mapping to individual files
    "paths": {
      // Bad: "@utils": ["src/utils.ts"]
      // Good: "@utils": ["src/utils/index.ts"]
    }
  }
}
```

---

## 39.4 Optimizing tsconfig.json for Speed

Specific compiler options significantly impact performance.

### Performance-Focused Configuration

```json
{
  "compilerOptions": {
    // Module Resolution
    "moduleResolution": "bundler",    // Faster than "node" for modern projects
    "resolveJsonModule": true,
    
    // Skip unnecessary work
    "skipLibCheck": true,             // Don't check node_modules types (huge speedup)
    "skipDefaultLibCheck": true,      // Skip lib.*.d.ts checking
    
    // Emit optimization
    "declaration": false,             // Skip if you don't need .d.ts (dev builds)
    "declarationMap": false,
    "sourceMap": true,                // Keep for debugging
    
    // Type Checking Strictness vs Speed trade-offs
    "strict": true,                   // Keep strictness
    "noEmitOnError": false,           // Emit JS even with type errors (faster dev builds)
    
    // Avoid expensive features if not needed
    "exactOptionalPropertyTypes": false,  // Strict but slow check
    "noFallthroughCasesInSwitch": false,  // Nice to have, not critical
    
    // Incremental
    "incremental": true,
    "tsBuildInfoFile": ".cache/.tsbuildinfo"
  },
  "exclude": [
    "node_modules",
    "dist",
    "build",
    "**/*.test.ts",      // Exclude tests from main build (use separate tsconfig.test.json)
    "**/*.spec.ts",
    "coverage",
    ".cache"
  ]
}
```

### Separate Configs for Dev/Prod

**tsconfig.json (Development):**
```json
{
  "extends": "./tsconfig.base.json",
  "compilerOptions": {
    "declaration": false,
    "declarationMap": false,
    "sourceMap": true,
    "noEmitOnError": false,
    "incremental": true
  },
  "exclude": ["**/*.test.ts", "**/*.spec.ts"]
}
```

**tsconfig.prod.json (Production):**
```json
{
  "extends": "./tsconfig.base.json",
  "compilerOptions": {
    "declaration": true,
    "declarationMap": true,
    "sourceMap": true,
    "noEmitOnError": true,
    "incremental": false  // Clean build for production
  }
}
```

**tsconfig.test.json (Testing):**
```json
{
  "extends": "./tsconfig.base.json",
  "compilerOptions": {
    "declaration": false,
    "noEmit": true,       // Type check only for tests
    "types": ["jest", "node"]
  },
  "include": ["src/**/*", "tests/**/*"]
}
```

---

## 39.5 Build Tool Comparisons

Different tools offer trade-offs between type checking and transpilation speed.

### Native TypeScript Compiler (tsc)

```bash
# Full type check + emit
npx tsc

# Pros:
# - Full type checking
# - Accurate error reporting
# - Declaration file generation

# Cons:
# - Slow (single-threaded)
# - No minification
# - No tree-shaking
```

### esbuild (Fast Transpilation)

```bash
npm install -D esbuild

# Transpile only (no type checking)
npx esbuild src/index.ts --bundle --outfile=dist/index.js --platform=node --target=es2020
```

```javascript
// esbuild.config.js
const esbuild = require('esbuild');

esbuild.build({
  entryPoints: ['src/index.ts'],
  bundle: true,
  outfile: 'dist/index.js',
  platform: 'node',
  target: 'es2020',
  format: 'esm',
  
  // TypeScript specific
  loader: { '.ts': 'ts' },
  tsconfig: './tsconfig.json',
  
  // Speed optimizations
  minify: true,
  treeShaking: true,
  splitting: true,  // Code splitting for ES modules
  
  // Watch mode
  watch: process.env.NODE_ENV === 'development'
}).catch(() => process.exit(1));
```

**Comparison:**
- **tsc**: 15 seconds (full type check + emit)
- **esbuild**: 0.05 seconds (transpile only, no type check)

### SWC (Rust-based Compiler)

```bash
npm install -D @swc/core @swc/cli
```

**.swcrc:**
```json
{
  "jsc": {
    "parser": {
      "syntax": "typescript",
      "tsx": false,
      "decorators": true,
      "dynamicImport": true
    },
    "target": "es2020",
    "loose": false,
    "externalHelpers": false,
    "keepClassNames": true,
    "transform": {
      "legacyDecorator": true,
      "decoratorMetadata": true
    }
  },
  "module": {
    "type": "es6",
    "strict": false,
    "strictMode": true,
    "lazy": false,
    "noInterop": false
  },
  "minify": true,
  "sourceMaps": true
}
```

```bash
# Compile with SWC (20x faster than tsc)
npx swc src -d dist

# Watch mode
npx swc src -d dist -w
```

### Integration Strategy: Type Check + Transpile Separation

```json
// package.json scripts
{
  "scripts": {
    "type-check": "tsc --noEmit",           // Type checking only (fast feedback)
    "build:fast": "esbuild src/index.ts --bundle --outfile=dist/index.js",  // Transpile only
    "build:prod": "npm run type-check && esbuild src/index.ts --bundle --minify --outfile=dist/index.js",
    
    "dev": "concurrently \"npm run type-check:watch\" \"npm run build:fast:watch\"",
    "type-check:watch": "tsc --noEmit --watch",
    "build:fast:watch": "esbuild src/index.ts --bundle --watch --outfile=dist/index.js"
  }
}
```

**Vite Configuration (Uses esbuild under the hood):**

```typescript
// vite.config.ts
import { defineConfig } from 'vite';
import react from '@vitejs/plugin-react';

export default defineConfig({
  plugins: [react()],
  
  // esbuild options (transpilation only in dev)
  esbuild: {
    target: 'es2020',
    logOverride: { 'this-is-undefined-in-esm': 'silent' }
  },
  
  build: {
    // For production, can use tsc for declarations + esbuild for bundling
    sourcemap: true,
    minify: 'esbuild',
    
    rollupOptions: {
      // Tree shaking optimization
      treeshake: 'smallest'
    }
  },
  
  optimizeDeps: {
    // Pre-bundle dependencies for faster dev startup
    include: ['lodash-es', 'react', 'react-dom'],
    esbuildOptions: {
      target: 'es2020'
    }
  }
});
```

---

## 39.6 Type Checking vs Transpilation Separation

The fastest builds separate type checking (slow, for correctness) from transpilation (fast, for execution).

### Development Workflow

```bash
# Terminal 1: Type checker (background, provides errors in IDE)
npx tsc --noEmit --watch

# Terminal 2: Fast transpiler (rebuilds on file change)
npx esbuild src/index.ts --bundle --watch --outfile=dist/index.js

# Terminal 3: Application runner
npx nodemon dist/index.js
```

### Orchestration with Concurrently

```json
{
  "scripts": {
    "dev": "concurrently -k -p \"[{name}]\" -n \"TypeCheck,Build,Run\" -c \"cyan.bold,yellow.bold,green.bold\" \"npm run type-check:watch\" \"npm run build:watch\" \"npm run start:watch\"",
    
    "type-check:watch": "tsc --noEmit --watch --preserveWatchOutput",
    "build:watch": "esbuild src/index.ts --bundle --watch --outfile=dist/index.js",
    "start:watch": "nodemon --watch dist dist/index.js"
  },
  "devDependencies": {
    "concurrently": "^8.2.0"
  }
}
```

### Production Pipeline

```bash
#!/bin/bash
# build.sh - Production build with full type safety

set -e  # Exit on error

echo "🔍 Type checking..."
npx tsc --noEmit

echo "📦 Building with esbuild..."
npx esbuild src/index.ts \
  --bundle \
  --minify \
  --platform=node \
  --target=node20 \
  --outfile=dist/index.js \
  --metafile=dist/meta.json

echo "📋 Generating declaration files..."
npx tsc --declaration --emitDeclarationOnly --outDir dist/types

echo "✅ Build complete!"
```

---

## 39.7 Memory Optimization for Large Projects

Large TypeScript projects can exhaust Node.js memory limits. Strategies to reduce memory usage:

### Increasing Memory Limits

```bash
# Increase Node.js heap size (default is ~1.4GB)
node --max-old-space-size=4096 ./node_modules/.bin/tsc

# Or in package.json
{
  "scripts": {
    "build": "NODE_OPTIONS='--max-old-space-size=4096' tsc",
    "build:win": "set NODE_OPTIONS=--max-old-space-size=4096 && tsc"
  }
}
```

### Reducing Memory Footprint

```json
{
  "compilerOptions": {
    // Limit concurrent type checking operations
    "maxNodeModuleJsDepth": 0,  // Don't deep-check node_modules
    
    // Reduce type instantiation depth
    "noErrorTruncation": false,
    
    // Don't keep all files in memory
    "incremental": true,
    "tsBuildInfoFile": "./.cache/.tsbuildinfo"
  },
  "exclude": [
    "**/*.d.ts",  // Often unnecessary to check declaration files
    "node_modules",
    "coverage",
    "dist",
    "**/*.test.ts"
  ]
}
```

### File System Caching

```bash
# Use filesystem cache for large projects (experimental in newer TypeScript versions)
npx tsc --incremental --build --verbose

# Clear cache if corrupted
rm -f .tsbuildinfo
rm -rf tsconfig.tsbuildinfo
```

### Project Splitting

If a single project exceeds memory limits, split into project references:

```json
// Instead of one huge tsconfig.json
{
  "references": [
    { "path": "./src/client" },
    { "path": "./src/server" },
    { "path": "./src/shared" }
  ]
}
```

---

## 39.8 CI/CD Build Optimization

Optimizing builds in continuous integration reduces pipeline duration and costs.

### Caching Strategies

**GitHub Actions with Caching:**

```yaml
# .github/workflows/build.yml
name: Build

on: [push, pull_request]

jobs:
  build:
    runs-on: ubuntu-latest
    
    steps:
      - uses: actions/checkout@v4
      
      - uses: actions/setup-node@v4
        with:
          node-version: '20'
          cache: 'npm'
      
      # Cache TypeScript build info
      - name: Cache TypeScript compilation
        uses: actions/cache@v3
        with:
          path: |
            .tsbuildinfo
            **/.tsbuildinfo
            node_modules/.cache
          key: ${{ runner.os }}-tsc-${{ hashFiles('**/tsconfig.json') }}-${{ hashFiles('**/*.ts') }}
          restore-keys: |
            ${{ runner.os }}-tsc-${{ hashFiles('**/tsconfig.json') }}-
            ${{ runner.os }}-tsc-
      
      - run: npm ci
      
      # Type check (cached)
      - name: Type check
        run: npx tsc --noEmit --incremental
      
      # Build with fast transpiler
      - name: Build
        run: npm run build
      
      # Cache build output for other jobs
      - uses: actions/upload-artifact@v3
        with:
          name: build-output
          path: dist
```

**Docker Layer Caching:**

```dockerfile
# Multi-stage build with caching
FROM node:20-alpine AS deps
WORKDIR /app
COPY package*.json ./
RUN npm ci

FROM node:20-alpine AS builder
WORKDIR /app
COPY --from=deps /app/node_modules ./node_modules
COPY . .

# Type check (can be cached if source unchanged)
RUN npm run type-check

# Build
RUN npm run build

FROM node:20-alpine AS runner
WORKDIR /app
COPY --from=builder /app/dist ./dist
COPY --from=builder /app/node_modules ./node_modules
CMD ["node", "dist/index.js"]
```

### Parallel Jobs

```yaml
# .github/workflows/ci.yml
jobs:
  type-check:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      - uses: actions/setup-node@v4
        with:
          node-version: '20'
      - run: npm ci
      - run: npx tsc --noEmit  # Just type checking, no emit

  lint:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      - uses: actions/setup-node@v4
      - run: npm ci
      - run: npm run lint

  build:
    runs-on: ubuntu-latest
    needs: [type-check]  # Only build if type check passes
    steps:
      - uses: actions/checkout@v4
      - uses: actions/setup-node@v4
      - run: npm ci
      - run: npm run build:fast  # Fast transpile only
```

### Selective Testing and Building

```bash
# Only build changed packages in monorepo (using Nx or Turborepo)
npx turbo run build --filter=[HEAD^1]

# Affected tests only
npx jest --changedSince=origin/main --passWithNoTests
```

---

## 39.9 Chapter Summary and Exercises

### Chapter Summary

This chapter covered TypeScript compilation performance optimization:

**Key Concepts:**

1. **Understanding Bottlenecks**: Type checking is the slowest phase (often 60-70% of build time). Use `--extendedDiagnostics` and `generateTrace` to identify specific slow types or files.

2. **Project References**: Split large codebases into smaller projects with explicit dependencies. Enables incremental compilation where only changed projects and their dependents are rebuilt.

3. **Incremental Compilation**: The `.tsbuildinfo` file tracks file hashes and dependencies. Subsequent builds skip unchanged files. Essential for large projects.

4. **tsconfig Optimization**: `skipLibCheck: true` provides massive speedups by skipping node_modules type checking. Separate configs for dev (no emit, fast feedback) and prod (full declarations).

5. **Build Tools**: 
   - **tsc**: Full type checking, accurate but slow
   - **esbuild/SWC**: 10-100x faster transpilation but no type checking
   - **Best practice**: Use esbuild/SWC for development builds, tsc for production type checking

6. **Type/Transpile Separation**: Run `tsc --noEmit --watch` for type errors in background while using esbuild for actual bundling. Fastest development workflow.

7. **Memory Management**: Use `--max-old-space-size` for large projects. Split into project references if hitting memory limits. Exclude test files and node_modules from main build.

8. **CI Optimization**: Cache `.tsbuildinfo` and `node_modules`. Run type checking, linting, and building in parallel jobs. Use fast transpilers for build artifacts after type check passes.

### Practical Exercises

**Exercise 1: Performance Analysis**

Set up a medium-sized TypeScript project and:
- Run `tsc --extendedDiagnostics` and document the timing breakdown
- Identify the slowest files using `generateTrace` + `analyze-trace`
- Measure impact of `skipLibCheck: true` on build time
- Compare build times: tsc vs esbuild vs SWC

**Exercise 2: Project References Setup**

Convert a monorepo to use project references:
- Split into `shared`, `server`, and `client` packages
- Configure `composite: true` and references between them
- Measure incremental build time vs full build time
- Set up a "solution" tsconfig.json at root

**Exercise 3: Development Workflow**

Create a fast development environment:
- Configure `tsc --noEmit --watch` for type checking
- Configure esbuild for bundling with watch mode
- Use `concurrently` to run both with colorized output
- Measure feedback loop time (file change to updated running code)

**Exercise 4: CI Pipeline Optimization**

Design a GitHub Actions workflow:
- Cache TypeScript build info between runs
- Run type check, lint, and build in parallel jobs
- Use fast transpilation for build artifacts
- Only generate declaration files on main branch builds
- Implement "affected" builds (only build changed packages)

**Exercise 5: Memory Optimization**

Take a large TypeScript codebase and:
- Measure baseline memory usage during build
- Implement strategies to reduce memory (exclude patterns, project splitting)
- Configure increased heap size if needed
- Document memory usage before and after optimizations

### Additional Resources

- **TypeScript Performance Wiki**: https://github.com/microsoft/TypeScript/wiki/Performance
- **esbuild Documentation**: https://esbuild.github.io/
- **SWC Documentation**: https://swc.rs/
- **Turborepo**: https://turbo.build/ for monorepo build optimization
- **Nx**: https://nx.dev/ for monorepo task running and caching

---

## Coming Up Next: Chapter 40 - Bundle Size Optimization

In the next chapter, we will explore optimizing the size of TypeScript application bundles:

- Tree shaking and dead code elimination
- Analyzing bundle composition
- Dynamic imports and code splitting
- Minification strategies
- Import optimization (barrel files, selective imports)
- Third-party library size management
- Compression and delivery optimization
- Runtime performance vs bundle size trade-offs

As applications grow, bundle size directly impacts load times, parse times, and runtime performance. Understanding how to optimize TypeScript output for production delivery is crucial for user experience and application performance.